In [ ]:
# get individual alpha ranges
import os
import scipy
from reftep_util_funcs import *

determine_freq_ranges = True
if determine_freq_ranges:
    alpha_default = 10.5
    freq_range_names = ['theta','alpha','beta','gamma']
    for site in ['Aalto','Tuebingen']:
            source_site = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
            features_site = rf"D:\REFTEP_ALL\Features_v2\Features_{site}"
            for subject in os.listdir(source_site):
                sourcepath_subject = os.path.join(source_site,subject)
                featurepath_subject = os.path.join(features_site,subject)
                os.makedirs(featurepath_subject, exist_ok=True)
                epochs_filepath = os.path.join(sourcepath_subject,f'{subject}_final_eeg-epo.fif')
                epochs = mne.read_epochs(epochs_filepath,verbose=False).crop(tmin=-1.4, tmax=-0.015) #use more data to estimate peak alpha freq, not from -1.015 till -0.015
                #estimate/determine alpha peak
                print(subject)
                alpha_peak, freqs = get_individual_alpha_peak(epochs,fmin=7, fmax=14, bw_scale=1.5) #get narrow bins
                alpha_peak_now = alpha_peak if alpha_peak is not False else alpha_default
                #define frequency ranges based on the alpha peak and save the frequency ranges
                _, freq_range_dict = get_freq_ranges_based_on_alpha_peak(alpha_peak_now,freq_range_names)
                freq_range_dict['alpha_peak'] = alpha_peak_now #add alpha peak info to dict
                is_default = True if alpha_peak is False else False
                freq_range_dict['alpha_peak_is_default'] = is_default #add alpha peak info to dict whether it is default or not
                print(subject, freq_range_dict)
                print(freqs)
                #save the info as .npy and .mat files
                freq_range_dict_path = os.path.join(featurepath_subject,f'{subject}-freq_ranges_dict')
                np.save(freq_range_dict_path,freq_range_dict)
                freq_range_dict_path_mat = os.path.join(featurepath_subject,f'{subject}-freq_ranges_dict_matlab.mat')
                scipy.io.savemat(freq_range_dict_path_mat,freq_range_dict)

In [ ]:
import os
import numpy as np
features_path = r"D:\REFTEP_ALL\Features_v2"
alpha_peaks_all = []
n_default_alpha_peaks = 0
subs = 0
for site_dir in ['Features_Aalto','Features_Tuebingen']:
    if "Features" in site_dir:
        site_path = os.path.join(features_path,site_dir)
        for subject in os.listdir(site_path):
            subs += 1
            fr_dict = np.load(os.path.join(site_path,subject,f'{subject}-freq_ranges_dict.npy'),allow_pickle=True).item()
            alpha_peak = fr_dict['alpha_peak']
            if not fr_dict['alpha_peak_is_default']:
                alpha_peaks_all.append(alpha_peak)
            else:
                n_default_alpha_peaks += 1
print(subs)
print(np.mean(alpha_peaks_all))
print(np.std(alpha_peaks_all))
print(np.min(alpha_peaks_all))
print(np.max(alpha_peaks_all))
print(n_default_alpha_peaks)

In [ ]:
for fr_name in ['theta','alpha','beta','gamma']:
    mins = []
    maxs = []
    for site_dir in ['Features_Aalto','Features_Tuebingen']:
        if "Features" in site_dir:
            site_path = os.path.join(features_path,site_dir)
            for subject in os.listdir(site_path):
                subs += 1
                fr_dict = np.load(os.path.join(site_path,subject,f'{subject}-freq_ranges_dict.npy'),allow_pickle=True).item()
                mins.append(fr_dict[fr_name][0])
                maxs.append(fr_dict[fr_name][1])
    print(fr_name)
    print(round(np.mean(mins),1))
    print(round(np.mean(maxs),1))
